In [3]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("/content/kg_dataset (1).csv")

In [5]:
df = df.dropna().drop_duplicates()

In [6]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  #many spaces
    text = text.strip()               #spaces at start and end
    return text

df["Text"] = df["Text"].apply(clean_text)
df["headline"] = df["headline"].apply(clean_text)

In [7]:
MAX_LEN = 512
df = df[df["Text"].str.len() < MAX_LEN]

In [8]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["Text"].tolist(),
    df["headline"].tolist(),
    test_size=0.1,
    random_state=42
)

print("Train size:", len(train_texts))
print("Validation size:", len(val_texts))

Train size: 1555
Validation size: 173


In [9]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import torch

#model
model_name = "google/mt5-small"

tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
#max input characters and max output
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 52

#training tokenization
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=MAX_INPUT_LENGTH)
train_targets = tokenizer(train_labels, truncation=True, padding=True, max_length=MAX_TARGET_LENGTH)

print(train_texts[:5])
print(train_labels[:5])

for i in range(5):
    print("LABEL", i, ":", tokenizer.decode(train_targets["input_ids"][i]))

#validation tokenization
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=MAX_INPUT_LENGTH)
val_targets = tokenizer(val_labels, truncation=True, padding=True, max_length=MAX_TARGET_LENGTH)

['Бүгүн,5-декабрда Жогорку КеңешРавшанбек Сабировдуэмгек, социалдык коргоо жана миграция министри кызматына дайындоого макулдук берген. Депутаттар анын талапкерлигине талкуусуз добуш беришти. 77 депутат «макул» деп добуш берди. Жалпы 83 депутат катталган. Кечээ, 4-декабрдаЖылдыз Полотоваэмгек, социалдык камсыздоо жана миграция министри кызматынан бошотулду.Равшанбек Сабировэмгек, социалдык камсыздоо жана миграция министринин милдетин аткаруучу болуп дайындалган.', 'Билим берүү жана илим министрлиги мектептер үчүн мамлекеттик электрондук күндөлүктү түзүү иштери аяктап калганын билдирди. Министрлик тарабынан иштелип чыккан электрондук күндөлүк окуучулар жана алардын ата-энелери, мектептер үчүн акысыз болот.', 'Бүгүн,27-августта окумуштуу, академикИлгиз Төрөкулович Айтматовдүйнөдөн кайтты. Бул тууралуу анын кызыЖамиля Айтматовабилдирди. Илгиз Айтматов— улуу жазуучуЧыңгыз Айтматовдуниниси. Ал 1931-жылы 8-февралда Фрунзе шаарында туулган. Көп жыл бою Кыргыз улуттук академиясында эмгектенген

In [11]:
class KyrgyzHeadlineDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, targets):
        self.encodings = encodings
        self.targets = targets

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        labels = self.targets["input_ids"][idx]
        #all pads to -100
        labels = [label if label != tokenizer.pad_token_id else -100 for label in labels]

        item = {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
            "labels": torch.tensor(labels),
        }
        return item

In [12]:
train_dataset = KyrgyzHeadlineDataset(train_encodings, train_targets)
val_dataset = KyrgyzHeadlineDataset(val_encodings, val_targets)

In [13]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [14]:
!pip install datasets
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5cb4b1e93b334d2a31b3e2c3cf5b80a0b345a73bc1138ba37587497595525e73
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [15]:
import evaluate

rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")

In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Prepare BLEU format: list of lists of tokens
    bleu_preds = [pred.split() for pred in decoded_preds]
    bleu_refs = [[label.split()] for label in decoded_labels]

    rouge_result = rouge_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    bleu_result = bleu_metric.compute(predictions=bleu_preds, references=bleu_refs)

    return {
        "rouge1": rouge_result["rouge1"].mid.fmeasure,
        "rouge2": rouge_result["rouge2"].mid.fmeasure,
        "rougeL": rouge_result["rougeL"].mid.fmeasure,
        "bleu": bleu_result["bleu"],
    }

In [17]:
import torch
print(torch.cuda.is_available())

True


In [18]:
print("Train dataset size:", len(train_dataset))

Train dataset size: 1555


In [19]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=100,
    do_eval=True,
    save_total_limit=2,
    predict_with_generate=True,
    report_to="none", #wandb off
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-19-7c39ad5a24b9>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,16.743600
200,7.578400
300,4.849600
400,4.049300
500,3.595900
600,3.406200
700,3.210100
800,3.160700
900,2.946400
1000,2.978400


TrainOutput(global_step=1950, training_loss=4.017389377691807, metrics={'train_runtime': 2008.16, 'train_samples_per_second': 7.743, 'train_steps_per_second': 0.971, 'total_flos': 3629267653632000.0, 'train_loss': 4.017389377691807, 'epoch': 10.0})

In [20]:
trainer.save_model("headline_model")
tokenizer.save_pretrained("headline_model")

('headline_model/tokenizer_config.json',
 'headline_model/special_tokens_map.json',
 'headline_model/spiece.model',
 'headline_model/added_tokens.json')

In [21]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

model = MT5ForConditionalGeneration.from_pretrained("headline_model").to("cuda")
tokenizer = MT5Tokenizer.from_pretrained("headline_model")

text = "Кыргыздын баатыр жигити Чолпонбай Түлөбердиев тууралуу тасма тартуу үчүн көптөгөн архивдик документтерди изилдеп, Таластагы анын туулуп-өскөн айылына жана Россиядагы мүрзөсүнө да бардык. 1942-жылдын 6-августунда катардагы жоокер Чолпонбай Түлөбердиев Дон дарыясынын жээгинде баатырларча курман болгон. Анын ысымы эркиндиктин символуна айланып, эрдиги Улуу Ата Мекендик согуштун тарыхынын ажырагыс бөлүгү болуп калды. Андыктан баатырдын өмүрүндөгү урунттуу учурларды баяндоону туура көрдүк."

# Указываем max_length явно
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to("cuda")

# Генерация заголовка
summary = model.generate(
    **inputs,
    max_length=64,
    num_beams=4,
    early_stopping=True
)

print(tokenizer.decode(summary[0], skip_special_tokens=True))

Бишкекте баатыр жигити Чолпонбай Түлөбердиев тууралуу тасма тартуу үчүн көптөгөн архивдик документтерди изилдеп жатат


In [24]:
text2 = "Кыргызстандын тоолуу аймактарында суук аба ырайы сакталып турат. Синоптиктердин маалыматына караганда, аптанын аягында температура кескин төмөндөп, айрым жерлерде кар жаашы мүмкүн. Өзгөчө кырдаалдар министрлиги жарандарды сак болууга чакырды."

# Указываем max_length явно
inputs = tokenizer(text2, return_tensors="pt", truncation=True, max_length=512).to("cuda")

# Генерация заголовка
summary = model.generate(
    **inputs,
    max_length=64,
    num_beams=4,
    early_stopping=True
)

print(tokenizer.decode(summary[0], skip_special_tokens=True))

Бишкекте суук аба ырайы сакталып турат


In [25]:
text3 = "Бишкекте жайгашкан IT-FEST көргөзмөсүнө жүздөгөн жаштар катышты. Иш-чарада инновациялык долбоорлор көрсөтүлүп, жаңы технологиялар боюнча презентациялар өткөрүлдү. Уюштуруучулардын айтымында, бул фестиваль жаштардын потенциалын ачууга багытталган."

# Указываем max_length явно
inputs = tokenizer(text3, return_tensors="pt", truncation=True, max_length=512).to("cuda")

# Генерация заголовка
summary = model.generate(
    **inputs,
    max_length=64,
    num_beams=4,
    early_stopping=True
)

print(tokenizer.decode(summary[0], skip_special_tokens=True))

Бишкекте жайгашкан IT-FEST көргөзмөсүнө жүздөгөн жаштар катышты


In [27]:
text3 = "2025-жылдын май айында Ала-Тоо Эл аралык университетинин IT факультетинде бүтүрүүчүлөр үчүн атайын практикалык экзамен уюштурулду. Бул экзаменде студенттер өздөрүнүн бүтүрүүчү долбоорлорун коргошту. Программалык камсыздоодон баштап, машиналык үйрөнүү жана жасалма интеллектке чейинки темалар камтылды. Жыйынтыгында, мыкты деп табылган долбоорлор атайын комиссия тарабынан белгиленип, кээ бир студенттерге стартаптык колдоо көрсөтүлөт. Университеттин деканы билдиргендей, мындай экзамендер студенттерди реалдуу дүйнөгө даярдоодо чоң роль ойнойт. Студенттердин даярдыгы жогорку деңгээлде болуп, көптөгөн иштер коомдук мааниге ээ экендиги белгиленди."

# Указываем max_length явно
inputs = tokenizer(text3, return_tensors="pt", truncation=True, max_length=512).to("cuda")

# Генерация заголовка
summary = model.generate(
    **inputs,
    max_length=64,
    num_beams=4,
    early_stopping=True
)

print(tokenizer.decode(summary[0], skip_special_tokens=True))

Бишкекте бүтүрүүчүлөр үчүн атайын экзамен уюштурулду


In [34]:
val_dataset = KyrgyzHeadlineDataset(val_encodings, val_targets)